<a href="https://colab.research.google.com/github/pradyumnac26/Localization-using-IMU-sensors-and-python/blob/main/Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import math
import csv

dates = []
dates1 = []

with open('Book1.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        dates.append(row[0])
        dates1.append(row[1])

#print(dates)

distarr = [float(i) for i in dates]
thetaarr = [int(i) for i in dates1]
print(distarr)
print(thetaarr)



global xcoord




def initMap():
    pi   = 3.14159265358979
    sm_a = 6378137.0
    sm_b = 6356752.314
    sm_EccSquared  = 6.69437999013e-03
    UTMScaleFactor = 0.9996
    lat_init  = 13.0691824 
    long_init = 77.5931928  
    myLatLng  = {"lat":lat_init, "lng":long_init} #decimal degree notation (not deg/min/sec)
    global xcoord,ycoord
    zonecoord = 0
    print(myLatLng)
    lattobemapped = 0 
    longtobemapped = 0 



    def updatedMarker():
        myLatLng1 = {"lat":lattobemapped, "lng": longtobemapped}
        #print(myLatLng1)
  
    """    
    * FootpointLatitude
    *
    * Computes the footpoint latitude for use in converting transverse
    * Mercator coordinates to ellipsoidal coordinates.
    *
    * Reference: Hoffmann-Wellenhof, B., Lichtenegger, H., and Collins, J.,
    *   GPS: Theory and Practice, 3rd ed.  New York: Springer-Verlag Wien, 1994.
    *
    * Inputs:
    *   y - The UTM northing coordinate, in meters.
    *
    * Returns:
    *   The footpoint latitude, in radians.
    *
    """
    def FootpointLatitude (y):
        n = (sm_a - sm_b) / (sm_a + sm_b)
        alpha_ = ((sm_a + sm_b) / 2.0)* (1 + (math.pow (n, 2.0) / 4) + (math.pow (n, 4.0) / 64));
        y_ = y / alpha_
        
        beta_ = (3.0 * n / 2.0) + (-27.0 * math.pow (n, 3.0) / 32.0)+ (269.0 * math.pow (n, 5.0) / 512.0)
        gamma_ = (21.0 * math.pow (n, 2.0) / 16.0) + (-55.0 * math.pow (n, 4.0) / 32.0)
        	
        delta_ = (151.0 * math.pow (n, 3.0) / 96.0) + (-417.0 * math.pow (n, 5.0) / 128.0)
        	
        epsilon_ = (1097.0 * math.pow (n, 4.0) / 512.0)
        	
        result = y_ + (beta_ * math.sin (2.0 * y_))+ (gamma_ * math.sin (4.0 * y_) + (delta_ * math.sin (6.0 * y_))  + (epsilon_ * math.sin (8.0 * y_)))
        return result 

"""
    * MapXYToLatLon
    *
    * Converts x and y coordinates in the Transverse Mercator projection to
    * a latitude/longitude pair.  Note that Transverse Mercator is not
    * the same as UTM; a scale factor is required to convert between them.
    *
    * Reference: Hoffmann-Wellenhof, B., Lichtenegger, H., and Collins, J.,
    *   GPS: Theory and Practice, 3rd ed.  New York: Springer-Verlag Wien, 1994.
    *
    * Inputs:
    *   x - The easting of the point, in meters.
    *   y - The northing of the point, in meters.
    *   lambda0 - Longitude of the central meridian to be used, in radians.
    *
    * Outputs:
    *   philambda - A 2-element containing the latitude and longitude
    *               in radians.
    *
    * Returns:
    *   The function does not return a value.
    *
    * Remarks:
    *   The local variables Nf, nuf2, tf, and tf2 serve the same purpose as
    *   N, nu2, t, and t2 in MapLatLonToXY, but they are computed with respect
    *   to the footpoint latitude phif.
    *
    *   x1frac, x2frac, x2poly, x3poly, etc. are to enhance readability and
    *   to optimize computations."""
    def MapXYToLatLon (x, y, lambda0, philambda):
        phif = FootpointLatitude (y)
        	
     
        ep2 = (math.pow (sm_a, 2.0) - math.pow (sm_b, 2.0))/ math.pow (sm_b, 2.0)
        	
        cf = math.cos (phif)
        	
        nuf2 = ep2 * math.pow (cf, 2.0)
        	
        Nf = math.pow (sm_a, 2.0) / (sm_b * math.sqrt (1 + nuf2))
        Nfpow = Nf
        	
        tf = math.tan (phif)
        tf2 = tf * tf
        tf4 = tf2 * tf2
        
        x1frac = 1.0 / (Nfpow * cf)
        
        Nfpow *= Nf
        x2frac = tf / (2.0 * Nfpow)
        
        Nfpow *= Nf
        x3frac = 1.0 / (6.0 * Nfpow * cf)
        
        Nfpow *= Nf
        x4frac = tf / (24.0 * Nfpow)
        
        Nfpow *= Nf;  
        x5frac = 1.0 / (120.0 * Nfpow * cf)
        
        Nfpow *= Nf
        x6frac = tf / (720.0 * Nfpow)
        
        Nfpow *= Nf
        x7frac = 1.0 / (5040.0 * Nfpow * cf)
        
        Nfpow *= Nf
        x8frac = tf / (40320.0 * Nfpow)
        
        x2poly = -1.0 - nuf2
        
        x3poly = -1.0 - 2 * tf2 - nuf2
        
        x4poly = 5.0 + 3.0 * tf2 + 6.0 * nuf2 - 6.0 * tf2 * nuf2 - 3.0 * (nuf2 *nuf2) - 9.0 * tf2 * (nuf2 * nuf2)
        
        x5poly = 5.0 + 28.0 * tf2 + 24.0 * tf4 + 6.0 * nuf2 + 8.0 * tf2 * nuf2
        
        x6poly = -61.0 - 90.0 * tf2 - 45.0 * tf4 - 107.0 * nuf2 + 162.0 * tf2 * nuf2
        
        x7poly = -61.0 - 662.0 * tf2 - 1320.0 * tf4 - 720.0 * (tf4 * tf2)
        
        x8poly = 1385.0 + 3633.0 * tf2 + 4095.0 * tf4 + 1575 * (tf4 * tf2)
        	
        philambda[0] = phif + x2frac * x2poly * (x * x) + x4frac * x4poly * math.pow (x, 4.0) + x6frac * x6poly * math.pow (x, 6.0) + x8frac * x8poly * math.pow (x, 8.0)
        	
        philambda[1] = lambda0 + x1frac * x + x3frac * x3poly * math.pow (x, 3.0) + x5frac * x5poly * math.pow (x, 5.0) + x7frac * x7poly * math.pow (x, 7.0)
        	
        return

    


    

    def btnToGeographic_OnClick (xx,yy,zoni):
        latlon = [0]*2
        x = xx
        y = yy
        zone = zoni
        southhemi = False
        UTMXYToLatLon (x, y, zone, southhemi, latlon)
        longtobemapped = RadToDeg (latlon[1])
        lattobemapped = RadToDeg (latlon[0])
        print("(",lattobemapped ," ,", longtobemapped,")")
        updatedMarker()

 
    
 
    def UTMXYToLatLon (x, y, zone, southhemi, latlon):
        x -= 500000.0
        x /= UTMScaleFactor
        if (southhemi) :
            y -= 10000000.0
        y /= UTMScaleFactor
        
        cmeridian = UTMCentralMeridian (zone)
        MapXYToLatLon (x, y, cmeridian, latlon)
        	
        return
          
    """ArcLengthOfMeridian
    *
    * Computes the ellipsoidal distance from the equator to a point at a
    * given latitude.
    *
    * Reference: Hoffmann-Wellenhof, B., Lichtenegger, H., and Collins, J.,
    * GPS: Theory and Practice, 3rd ed.  New York: Springer-Verlag Wien, 1994.
    *
    * Inputs:
    *     phi - Latitude of the point, in radians.
    *
    * Globals:
    *     sm_a - Ellipsoid model major axis.
    *     sm_b - Ellipsoid model minor axis.
    *
    * Returns:
    *     The ellipsoidal distance of the point from the equator, in meters.
    *
    """
    def ArcLengthOfMeridian (phi):
        n = (sm_a - sm_b) / (sm_a + sm_b)
        alpha = ((sm_a + sm_b) / 2.0)* (1.0 + (math.pow (n, 2.0) / 4.0) + (math.pow (n, 4.0) / 64.0))

        beta = (-3.0 * n / 2.0) + (9.0 * math.pow (n, 3.0) / 16.0) + (-3.0 * math.pow (n, 5.0) / 32.0)

        gamma = (15.0 * math.pow (n, 2.0) / 16.0) + (-15.0 * math.pow (n, 4.0) / 32.0)
    
        
        delta = (-35.0 * math.pow (n, 3.0) / 48.0) + (105.0 * math.pow (n, 5.0) / 256.0)
    
      
        epsilon = (315.0 * math.pow (n, 4.0) / 512.0)
        result = alpha* (phi + (beta * math.sin (2.0 * phi))+ (gamma * math.sin (4.0 * phi))+ (delta * math.sin (6.0 * phi))	+ (epsilon * math.sin (8.0 * phi)))

        return result

 

 """    
    * UTMCentralMeridian
    *
    * Determines the central meridian for the given UTM zone.
    *
    * Inputs:
    *     zone - An integer value designating the UTM zone, range [1,60].
    *
    * Returns:
    *   The central meridian for the given UTM zone, in radians, or zero
    *   if the UTM zone parameter is outside the range [1,60].
    *   Range of the central meridian is the radian equivalent of [-177,+177].
    *
    """
    def UTMCentralMeridian (zone):
        cmeridian = DegToRad (-183.0 + (zone * 6.0))
        return cmeridian

"""    
    * MapLatLonToXY
    *
    * Converts a latitude/longitude pair to x and y coordinates in the
    * Transverse Mercator projection.  Note that Transverse Mercator is not
    * the same as UTM; a scale factor is required to convert between them.
    *
    * Reference: Hoffmann-Wellenhof, B., Lichtenegger, H., and Collins, J.,
    * GPS: Theory and Practice, 3rd ed.  New York: Springer-Verlag Wien, 1994.
    *
    * Inputs:
    *    phi - Latitude of the point, in radians.
    *    lambda - Longitude of the point, in radians.
    *    lambda0 - Longitude of the central meridian to be used, in radians.
    *
    * Outputs:
    *    xy - A 2-element array containing the x and y coordinates
    *         of the computed point.
    *
    * Returns:
    *    The function does not return a value.
    *
    """
    def MapLatLonToXY (phi, lambda1, lambda0, xy):
        ep2 = (math.pow (sm_a, 2.0) - math.pow (sm_b, 2.0)) / math.pow (sm_b, 2.0)
        nu2 = ep2 * math.pow (math.cos (phi), 2.0)
    
        N = math.pow (sm_a, 2.0) / (sm_b * math.sqrt (1 + nu2))
    
        t = math.tan (phi)
        t2 = t * t
        tmp = (t2 * t2 * t2) - math.pow (t, 6.0)

        l = lambda1-lambda0
    """
         Precalculate coefficients for l**n in the equations below
           so a normal human being can read the expressions for easting
           and northing
           -- l**1 and l**2 have coefficients of 1.0 """
        l3coef = 1.0 - t2 + nu2
    
        l4coef = 5.0 - t2 + 9 * nu2 + 4.0 * (nu2 * nu2)
    
        l5coef = 5.0 - 18.0 * t2 + (t2 * t2) + 14.0 * nu2 - 58.0 * t2 * nu2
    
        l6coef = 61.0 - 58.0 * t2 + (t2 * t2) + 270.0 * nu2 - 330.0 * t2 * nu2
    
        l7coef = 61.0 - 479.0 * t2 + 179.0 * (t2 * t2) - (t2 * t2 * t2)
    
        l8coef = 1385.0 - 3111.0 * t2 + 543.0 * (t2 * t2) - (t2 * t2 * t2)
    
        xy[0] = N * math.cos (phi) * l + (N / 6.0 * math.pow (math.cos (phi), 3.0) * l3coef * math.pow (l, 3.0))  + (N / 120.0 * math.pow (math.cos (phi), 5.0) * l5coef * math.pow (l, 5.0))+ (N / 5040.0 * math.pow (math.cos (phi), 7.0) * l7coef * math.pow (l, 7.0))
    
        xy[1] = ArcLengthOfMeridian (phi)  + (t / 2.0 * N * math.pow (math.cos (phi), 2.0) * math.pow (l, 2.0)) + (t / 24.0 * N * math.pow (math.cos (phi), 4.0) * l4coef * math.pow (l, 4.0)) + (t / 720.0 * N * math.pow (math.cos (phi), 6.0) * l6coef * math.pow (l, 6.0))+ (t / 40320.0 * N * math.pow (math.cos (phi), 8.0) * l8coef * math.pow (l, 8.0))
    
        return

# converts degrees to radians
    def DegToRad(deg): 
        return (deg / 180.0 * pi)
		
	#converts radians to degrees
    def RadToDeg(rad):
        return (rad / pi * 180.0)
    
    """
    
    
    * LatLonToUTMXY
    *
    * Converts a latitude/longitude pair to x and y coordinates in the
    * Universal Transverse Mercator projection.
    *
    * Inputs:
    *   lat - Latitude of the point, in radians.
    *   lon - Longitude of the point, in radians.
    *   zone - UTM zone to be used for calculating values for x and y.
    *          If zone is less than 1 or greater than 60, the routine
    *          will determine the appropriate zone from the value of lon.
    *
    * Outputs:
    *   xy - A 2-element array where the UTM x and y values will be stored.
    *
    * Returns:
    *   The UTM zone used for calculating the values of x and y.
    *
    """
    def LatLonToUTMXY (lat, lon, zone, xy):
        MapLatLonToXY (lat, lon, UTMCentralMeridian (zone), xy)
        xy[0] = xy[0] * UTMScaleFactor + 500000.0
        xy[1] = xy[1] * UTMScaleFactor
        if (xy[1] < 0.0):
            xy[1] = xy[1] + 10000000.0
        return zone    

    def btnToUTM_OnClick (latitude,longitude) :
        xy = [0]*2
    
        lon = longitude
        if ((lon < -180.0) or (180.0 <= lon)) :
            print("The longitude you entered is out of range.  " + "Please enter a number in the range [-180, 180).")
            return False
        

        lat = latitude

        if ((lat < -90.0) or (90.0 < lat)) :
            print("The latitude you entered is out of range.  " +
                   "Please enter a number in the range [-90, 90].")
            return False
        

        
        zone = math.floor ((lon + 180.0) / 6) + 1

        zone = LatLonToUTMXY (DegToRad (lat), DegToRad (lon), zone, xy)
        xcoord = xy[0]
        ycoord = xy[1]
       
        zonecoord = zone
        
        print(zonecoord)
        return xcoord,ycoord,zonecoord
    
   
   
    xcoord,ycoord,zonecoord = btnToUTM_OnClick (lat_init,long_init)



    #distarr = [3,6,9,12,15,18,21,24,27,30,33,36,39,42,45,48,51,54,57,60]
    #thetaarr= [170,190,190,192,176,175,165,164,169,175,177,183,184,188,197,178,184,192,270,277]
    
    #filtering
    y=0
    for x in range(len(distarr)): # to eliminate missed magnetic cuts
        if distarr[x]>0 :
            distarr[y] = distarr[x]
            thetaarr[y] = thetaarr[x]
            y+=1
    for i in range(len(distarr)): # magnetometer algorithm for turnings
        if (i==0 or i>0):
            if (thetaarr[i]>=45 and thetaarr[i]<=135 )  : # approximating 90 as and when it is between 45 and 135 
                thetaarr[i] = 90
			
            elif (thetaarr[i]<=45) :
                thetaarr[i] = 0
			
            elif (thetaarr[i]>=135 and thetaarr[i]<=225) :
                thetaarr[i] = 180
			
            elif (thetaarr[i]>=225 and thetaarr[i]<=315) :
                thetaarr[i] = 270
			
            elif (thetaarr[i]>=315) :
                thetaarr[i] = 360


        if (i==0) :
            xupdated = xcoord - ((distarr[i])*math.cos((thetaarr[i])*(3.14/180)))
            yupdated = ycoord - ((distarr[i])*math.sin((thetaarr[i])*(3.14/180)))
            btnToGeographic_OnClick(xupdated,yupdated,zonecoord)
            xcoord = xupdated
            ycoord = yupdated
        else :
            xupdated = xcoord - ((distarr[i]-distarr[i-1])*math.cos((thetaarr[i])*(3.14/180)))
            yupdated = ycoord - ((distarr[i]-distarr[i-1])*math.sin((thetaarr[i])*(3.14/180)))
            btnToGeographic_OnClick(xupdated,yupdated,zonecoord)
            xcoord = xupdated
            ycoord = yupdated

    

if __name__=="__main__":
  initMap()



[3.14, 6.28, 9.42, 12.0, 15.0, 18.0, 21.0, 24.0, 27.0, 30.0, 33.0, 36.0, 39.0, 42.0, 45.0, 48.0, 51.0, 53.0, 57.0, 60.0]
[170, 190, 190, 192, 176, 175, 165, 164, 169, 175, 177, 183, 184, 188, 197, 178, 184, 192, 270, 277]
{'lat': 13.0691824, 'lng': 77.5931928}
43
( 13.069182064322055  , 77.59322173341185 )
( 13.069181728642448  , 77.5932506668466 )
( 13.069181392959617  , 77.59327960028064 )
( 13.069181117141369  , 77.59330337361126 )
( 13.069180796419726  , 77.59333101701834 )
( 13.06918047569514  , 77.59335866042478 )
( 13.069180154967603  , 77.59338630383058 )
( 13.069179834237115  , 77.59341394723576 )
( 13.069179513503682  , 77.59344159064024 )
( 13.069179192767297  , 77.5934692340441 )
( 13.069178872027965  , 77.5934968774473 )
( 13.069178551285683  , 77.59352452084985 )
( 13.069178230540455  , 77.59355216425176 )
( 13.069177909792275  , 77.59357980765303 )
( 13.069177589041145  , 77.59360745105361 )
( 13.069177268287065  , 77.5936350944536 )
( 13.069176947530039  , 77.5936627378